# 正则表达式符号与方法

## 常用符号

- 常用符号：点号，星号，问号与括号
- 常用方法：findall, search, sub
- 常用方法：.
    - . : 匹配任意字符，换行符\n除外
    - * ：匹配前一个字符0次或无限次
    - ? ：匹配前一个字符0次或1次
    - .*：贪心算法
    - .*?：非贪心算法
    - （）：括号内的数据作为结果返回


## 效果展示

- findall： 匹配所有符合规律的内容，返回包含结果的列表
- Search：匹配并提取第一个符合规律的内容，返回一个正则表达式对象（object)
- Sub：替换符合规律的内容，返回替换后的值

## 常用技巧

- import re    
- from re import * 
- from re import findall,search,sub,S
- 不需要complie
- 使用\d+匹配纯数字

# 正则表达式的应用举例

- 使用findall与search从大量文本中匹配感兴趣的内容
- 使用sub实现换页功能

## 匹配多段内容

- 灵活使用findall与search
- 先抓大再抓小

# 中文乱码与header问题

In [84]:
import requests
head = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.118 Safari/537.36'}
html = requests.get('http://jp.tingroom.com/yuedu/yd300p/',headers = head)
html.encoding = 'utf-8' #这一行是将编码转为utf-8否则中文会显示乱码。

# Post请求方式

In [85]:
url = 'https://www.crowdfunder.com/browse/deals&template=false'
data = {
    'entities_only':'true',
    'page':'2'
}
html_post = requests.post(url, data=data)
title = re.findall('"card-title">(.*?)</div>',html_post.text,re.S)
for each in title:
    pass

# 抓取极客学院课程信息

In [6]:
import requests
import re

In [79]:
class spider:
    def __init__(self):
        print("开始爬取内容")
        
    # 获取网页源代码
    def getsource(self, url):
        html = requests.get(url)
        return html.text
    
    # 用来生成不同页数的链接
    def change_page(self, url, total_page):
        now_page = int(re.search("pageNum=(\d+)", url, re.S).group(1))
        page_group = []
        for i in range(now_page, total_page + 1):
            link = re.sub("pageNum=\d+", "pageNum=%s" %i, url, re.S)
            page_group.append(link)
        return page_group
    
    # 抓取每个课程块的信息
    def get_every_class(self, source):
        everyclass = re.findall('(<li id=".*?</li>)', source, re.S)
        return everyclass
    
    # 从每个课程块中提取我们需要的信息
    def getinfo(self, eachclass):
        info = {}
        info["title"] = re.search('" class="lessonimg" title="(.*?)" alt="', eachclass, re.S).group(1).strip()
        info["content"] = re.search('<p style="height: 0px; opacity: 0; display: none;">(.*?)</p>', eachclass, re.S).group(1).strip()
        timeandlevel = re.findall('<em>(.*?)</em>', eachclass, re.S)
        info["classtime"] = timeandlevel[0].strip().replace("\t", "").replace("\n", " ")
        info["classlevel"] = timeandlevel[1].strip()
        info['learnnum'] = re.search('"learn-number">(.*?)</em>',eachclass, re.S).group(1).strip()
        return info
    
    # 保存结果到info.txt文件中
    def saveinfo(self, classinfo):
        with open("info.txt", "a", encoding="gbk") as f:
            for each in classinfo:
                f.writelines("title:" + each["title"] + "\n")
                f.writelines("content:" + each["content"] + "\n")
                f.writelines("classtime:" + each["classtime"] + "\n")
                f.writelines("classlevel:" + each["classlevel"] + "\n")
                f.writelines("learnnum:" + each["learnnum"] + "\n")

In [80]:
classinfo = []
url = "http://www.jikexueyuan.com/course/?pageNum=1"
jikespider = spider()
all_links = jikespider.change_page(url, 5)
for link in all_links:
    print("正在处理页面：" + link)
    html = jikespider.getsource(link)
    everyclass = jikespider.get_every_class(html)
    for each in everyclass:
        info = jikespider.getinfo(each)
        classinfo.append(info)
jikespider.saveinfo(classinfo)
print("内容爬取完成")

开始爬取内容
正在处理页面：http://www.jikexueyuan.com/course/?pageNum=1
正在处理页面：http://www.jikexueyuan.com/course/?pageNum=2
正在处理页面：http://www.jikexueyuan.com/course/?pageNum=3
正在处理页面：http://www.jikexueyuan.com/course/?pageNum=4
正在处理页面：http://www.jikexueyuan.com/course/?pageNum=5
内容爬取完成
